In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
import sys
from threading import Thread
from ipygany import PolyMesh, Scene, IsoColor, WarpByScalar
import pyvista as pv
import pyvistaqt as pvqt
import colorcet as cc # Colormaps:

# pv.set_jupyter_backend('pythreejs')
# pv.set_jupyter_backend('ipygany') # ipygany fails " Failed to use notebook backend: "
# pv.set_jupyter_backend('panel') # Doesn't work either, produces no output
import numpy as np
import h5py
import hdf5storage # conda install hdf5storage
from pathlib import Path
import bqplot.scales
import seaborn as sns
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize
# import mplcursors
import math # For color map generation
from matplotlib.colors import ListedColormap
from matplotlib.cm import hsv

import ipywidgets as widgets
# from PyQt5 import QtWidgets, uic
from pyvistaqt import QtInteractor, MainWindow

In [2]:
# import PhoPositionalData as pdp
# from PhoPositionalData import load_exported, process_data
from PhoPositionalData.load_exported import *
# from PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
from PhoPositionalData.process_data import *
from PhoPositionalData.plot_data import *
from PhoPositionalData.import_data import * # build_spike_positions_list, build_cellID_reverse_lookup_map

In [3]:
# NeuroPy (Diba Lab Python Repo) Loading
try:
    from neuropy import core
except ImportError:
    sys.path.append(r'C:\Users\Pho\repos\NeuroPy')
    # sys.path.append(r'/Users/pho/repo/Python Projects/NeuroPy')
    print('neuropy module not found, adding directory to sys.path. \nUpdated sys.path: {}'.format(sys.path))
    from neuropy import core
from neuropy.core.dataSession import DataSession, processDataSession

basedir = 'R:\data\Bapun\Day5TwoNovel'
# basedir = '/Volumes/iNeo/Data/Bapun/Day5TwoNovel'
sess = core.processDataSession(basedir)

neuropy module not found, adding directory to sys.path. 
Updated sys.path: ['C:\\Users\\Pho\\repos\\PhoPy3DPositionAnalysis2021', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\python39.zip', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\DLLs', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz', '', 'C:\\Users\\Pho\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib\\site-packages', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib\\site-packages\\win32', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Pho\\anaconda3\\envs\\phoviz\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Pho\\.ipython', 'C:\\Users\\Pho\\repos\\NeuroPy']
Issue with pickled POSIX_PATH on windows for path R:\data\Bapun\Day5TwoNovel\RatS-Day5TwoNovel-2020-12-04_07-55-09.probegroup.npy, falling back to non-pickled version...
li

In [9]:
# all_spikes = sess.neurons.get_all_spikes()
# all_spikes.shape # (16318817,)

# sess.neurons.spiketrains.shape # (109,)
# sess.neurons.spiketrains[0].shape # (280944,)
# sess.neurons.spiketrains[1].shape # (47282,)
# # sess.neurons.get_by_id(

# Filter the segments as desired
active_epoch_times = sess.epochs['maze1']  # array([11070, 13970], dtype=int64)

# spike_matrix, spike_cells, num_cells, spike_list, spike_positions_list, cell_ids, reverse_cellID_idx_lookup_map
active_epoch_session_Neurons = sess.neurons.get_neuron_type('pyr') # Filter by pyramidal cells only
active_epoch_session_Neurons = active_epoch_session_Neurons.time_slice(active_epoch_times[0], active_epoch_times[1]) # returns a core.Neurons object with its spiketrains filtered for the provided start/end times
num_cells = active_epoch_session_Neurons.n_neurons
spike_list = active_epoch_session_Neurons.spiketrains
cell_ids = active_epoch_session_Neurons.neuron_ids
# t, x, y
active_epoch_pos = sess.position.time_slice(active_epoch_times[0], active_epoch_times[1])
t = active_epoch_pos.time
x = active_epoch_pos.x
y = active_epoch_pos.y
# speeds = active_epoch_pos.speed # note this has 1 less element than active_epoch_pos.x

# Determine the x and y positions each spike occured for each cell
spike_positions_list = build_spike_positions_list(spike_list, t, x, y) 
 # reverse_cellID_idx_lookup_map: Allows reverse indexing into the linear imported array using the original cell ID indicies
reverse_cellID_idx_lookup_map = build_cellID_reverse_lookup_map(cell_ids)
    
print('num_cells: {}'.format(num_cells))
print('cell_ids: {}'.format(cell_ids)) # cell_ids is now a regular python list with 57 elements

num_cells: 52
cell_ids: [  1   3   7   8   9  10  11  13  14  17  20  23  27  28  34  37  38  39
  40  41  42  43  44  47  48  49  50  52  53  54  55  56  57  58  60  61
  62  63  64  65  68  70  75  77  78  79  80  87  91  93  95 107]


In [10]:
# reverse_cellID_idx_lookup_map[2]
active_cells_colormap = cc.rainbow[:num_cells]
# active_cells_colormap
# print('{}'.format(cell_ids[0, 0]))

In [11]:
# Split the position data into equal sized chunks to be displayed at a single time. These will look like portions of the trajectory and be used to animate. # Chunk the data to create the animation.
# curr_view_window_length = 150 # View 5 seconds at a time (30fps)
curr_view_window_length = 30 # View 5 seconds at a time (30fps)
# The original length 324574 / 30 = 10819
# t_fixedSegements,x_fixedSegements,y_fixedSegements,speeds_fixedSegements,dt_fixedSegements,dx_fixedSegements,dy_fixedSegements = process_chunk_equal_poritions_data(t, x, y, speeds, dt, dx, dy, curr_view_window_length)
# print('shapes - t_fixedSegements: {}, x_fixedSegements: {}, y_fixedSegements: {}'.format(np.shape(t_fixedSegements), np.shape(x_fixedSegements), np.shape(y_fixedSegements)))
# z_fixedSegements = np.zeros_like(x_fixedSegements)
# np.vstack(

data_vectors_matrix = np.vstack([t, x, y]) # pack the variables of interest into the data_vector_matrix
outputMatrix_fixedSegements = process_chunk_equal_poritions_data_vectors(data_vectors_matrix, curr_view_window_length)
print('shape - outputMatrix_fixedSegements: {}'.format(np.shape(outputMatrix_fixedSegements)))
num_data_vectors = np.shape(outputMatrix_fixedSegements)[0]
# unpack the result
t_fixedSegements, x_fixedSegements, y_fixedSegements = [np.squeeze(outputMatrix_fixedSegements[i,:,:]) for i in np.arange(num_data_vectors)]
print('shapes - t_fixedSegements: {}, x_fixedSegements: {}, y_fixedSegements: {}'.format(np.shape(t_fixedSegements), np.shape(x_fixedSegements), np.shape(y_fixedSegements)))
z_fixedSegements = np.zeros_like(y_fixedSegements)




shape - outputMatrix_fixedSegements: (3, 5800, 30)
shapes - t_fixedSegements: (5800, 30), x_fixedSegements: (5800, 30), y_fixedSegements: (5800, 30)


In [12]:
num_time_points = np.shape(x_fixedSegements)[0]
print('num_time_points: {}\n'.format(num_time_points))
animal_location_sphere = pv.Sphere(radius=2.3)
animal_location_direction_cone = pv.Cone()

#     animal_direction_arrow = pv.Arrow()
last_only_opacity_values = np.zeros([curr_view_window_length,])
last_only_opacity_values[-1] = 1.0

# last_only_opacity_values = np.zeros([1,curr_view_window_length])
# last_only_opacity_values[-1] = 1.0
gradually_fading_opacity_values = np.arange(curr_view_window_length)

# active_trail_opacity_values = last_only_opacity_values.copy()
active_trail_opacity_values = gradually_fading_opacity_values.copy()
# print('active_trail_opacity_values: {}\n'.format(np.shape(active_trail_opacity_values)))
    
## Slider with Callback Function Example:

# p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance

def build_flat_map_plot_data():
    # Builds the flat base maze map that the other data will be plot on top of
    z = np.zeros_like(x)
    # ic(x, y, z)
    point_cloud = np.vstack((x, y, z)).T
    # ic(point_cloud)
    pdata = pv.PolyData(point_cloud)
    pdata['occupancy heatmap'] = np.arange(np.shape(point_cloud)[0])
    # create many spheres from the point cloud
    # geo = pv.Sphere(radius=0.2, phi_resolution=3, theta_resolution=3)
    # geo = pv.ParametricSuperEllipsoid(n1=0.1, n2=2)
    # geo = pv.ParametricSuperEllipsoid(xradius=0.2, yradius=0.2, zradius=0.02, n1=0.1, n2=0.1) # makes a square title geometry
    geo = pv.Circle(radius=0.5)
    pc = pdata.glyph(scale=False, geom=geo)
    return pdata, pc

def on_slider_update_mesh(value):
    curr_i = int(value)
    # point_cloud_fixedSegements = np.vstack((x_fixedSegements, y_fixedSegements, z_fixedSegements)).T
    point_cloud_fixedSegements = np.column_stack((x_fixedSegements[curr_i,:], y_fixedSegements[curr_i,:], z_fixedSegements[curr_i,:]))
#     print('point_cloud_fixedSegements: {}\n'.format(np.shape(point_cloud_fixedSegements)))
    curr_animal_point = point_cloud_fixedSegements[-1,:] # Get the last point
#     actor = p.add_mesh(pyvista.Cube(), show_edges=True)
#     mesh = pv.PolyData(point_cloud_fixedSegements)
#     mesh.plot(point_size=10, style='points')
    pdata = pv.PolyData(point_cloud_fixedSegements) # a mesh
#     pdata['orig_sphere'] = np.arange(np.shape(point_cloud)[0])
    
#     print(last_only_opacity_values)
    
#     pdata.point_data['pho_fade_values'] = np.arange(np.shape(point_cloud_fixedSegements)[0])
    pdata.point_data['pho_fade_values'] = active_trail_opacity_values

    # create many spheres from the point cloud
    pc = pdata.glyph(scale=False, geom=animal_location_sphere)
    # pc.point_data['pho_fade_values'] = active_trail_opacity_values
        
    # plines = p.add_lines(point_cloud_fixedSegements, name='animal_trajectory_lines', color='orange')
    # plines.point_data['pho_fade_values'] = active_trail_opacity_values
    #     p.add_mesh(pc, name='animal_location', color="red", opacity='linear', scalars='pho_fade_values', nan_opacity=0.0)
    # p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity=0.8, nan_opacity=0.0)
    # p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity="pho_fade_values", nan_opacity=0.0)

    p.add_mesh(pc, name='animal_location', color='red', ambient=0.6, opacity=0.8, nan_opacity=0.0, render_lines_as_tubes=True)
#     p.add_mesh(animal_location_sphere, color="grey", ambient=0.6, opacity=0.5, show_edges=False)
    
#     # Create spline with 1000 interpolation points
#     spline = pv.Spline(point_cloud_fixedSegements)
# #     # add scalars to spline and plot it
#     spline["scalars"] = np.arange(spline.n_points)
#     tube = spline.tube(radius=0.1)
# #     tube.plot(smooth_shading=True)
#     p.add_mesh(tube, name='tube')

#     sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
#     p.add_mesh(sphere, name='sphere', show_edges=True)
    return


        
p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance
# p = pv.Plotter() # Use just like you would a pv.Plotter() instance
# p.background_color = 'black'
p.clear()

# p.add_slider_widget(on_slider_update_mesh, [0, (num_time_points-1)], title='Trajectory Timestep', event_type='always', style='modern')
# active_frame_range = np.arange(num_time_points) # All frames
active_frame_range = np.arange(num_time_points) # All frames
# active_frame_range = np.arange(100) # Only 100 frames
# active_frame_range = range(100) # Only 100 frames                  
pdata_maze, pc_maze = build_flat_map_plot_data()
# Plot the flat arena
# p.add_points(pdata)
p.add_mesh(pc_maze, name='maze_bg', color="black")
# make_mp4_from_plotter(p, active_frame_range, on_slider_update_mesh, filename='complete_plotting.mp4')
p.add_bounding_box()
p.show_grid()
p.add_axes(line_width=5, labels_off=True)
# p.hide_axes()
# p.show()
                  
print('all done!')

num_time_points: 5800

all done!
